- augmenter la RAM utilisée par SPARK !! : http://www.datastax.com/dev/blog/zen-art-spark-maintenance, http://www.datastax.com/dev/blog/common-spark-troubleshooting : **plus ou moins fait... à ajuster éventuellement**
- faire la table dans cassandra **fait : vérifier que le schéma est correct**
- dropper le poids ** oublié... à faire : changer le schéma de la table dans cqlsh et changer le code de chargement des DataFrame**
- ajouter une colonne date (full timestamp) ** fait **
- insérer fichier par fichier ? charger par batch de fichiers ? ** en test **
- possible de paralleliser ? ** je ne pense pas : idée ? **
- faire un test sur la vitesse de chargement : ** en cours **

----

## Vérification du bon fonctionnement de PySpark

In [1]:
print(sc)

In [2]:
sc.master

u'spark://172.31.20.22:7077'

In [3]:
rdd = sc.parallelize([1, 2, 3]).map(lambda x: x*2).filter(lambda x: x > 2).collect()

In [4]:
rdd

[4, 6]

---

## Chargement des données

In [6]:
from os import listdir
from os.path import isfile, join
import re
from time import time
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
sql = SQLContext(sc)

On récupère la liste des fichiers

In [65]:
directory = '/mnt/wikidata/wikistats/'
datafiles = [f for f in listdir(directory) 
             if isfile(join(directory, f)) and f.startswith('pagecounts')]
print('Il existe {} fichiers.'.format(len(datafiles)))

Il existe 2160 fichiers.


A chaque fichier on associer un datastamp :

In [68]:
def extract_date(datafile):
    rex = re.compile(r'pagecounts-(\d{4})(\d{2})(\d{2})-(\d{2}).*')
    s = rex.match(datafile)
    datetime = '{}-{}-{} {}:00'.format(*(s.group(i) for i in range(1, 5)))
    return datetime

In [69]:
datafiles_datetimes = [(datafile, extract_date(datafile)) for datafile in datafiles]

In [70]:
datafiles_datetimes[:10]

[('pagecounts-20110130-070000.gz', '2011-01-30 07:00'),
 ('pagecounts-20110305-210000.gz', '2011-03-05 21:00'),
 ('pagecounts-20110113-180000.gz', '2011-01-13 18:00'),
 ('pagecounts-20110313-220000.gz', '2011-03-13 22:00'),
 ('pagecounts-20110101-060000.gz', '2011-01-01 06:00'),
 ('pagecounts-20110314-110000.gz', '2011-03-14 11:00'),
 ('pagecounts-20110124-110000.gz', '2011-01-24 11:00'),
 ('pagecounts-20110116-220000.gz', '2011-01-16 22:00'),
 ('pagecounts-20110108-230000.gz', '2011-01-08 23:00'),
 ('pagecounts-20110109-060000.gz', '2011-01-09 06:00')]

On teste le premier fichier.

In [88]:
datafile = datafiles_datetimes[2][0]
datetime = datafiles_datetimes[2][1]
path='file:///mnt/wikidata/wikistats/' + datafile 

In [99]:
datafile

'pagecounts-20110101-060000.gz'

In [73]:
def create_row(row, datetime):
    projectcode, page, views, _ = row.split(' ')
    return [datetime, projectcode, page, int(views)]

In [74]:
def no_points(row):
    return '.' not in row[1]

In [90]:
rdd = sc.textFile(path).map(lambda r: create_row(r, datetime)).filter(no_points)
df = sql.createDataFrame(rdd, schema=['datetime', 'projectcode', 'page', 'views'])

In [91]:
t0 = time()
df.write.format("org.apache.spark.sql.cassandra").\
        options(table="wikidata", keyspace="projet").\
        save(mode="append")
t1 = time()

In [50]:
df.write.format("org.apache.spark.sql.cassandra").\
    options(table="wikipediadata", keyspace="projet").\
    save(mode="append")

On time l'insertion de 10 fichiers via le master:

In [52]:
ts = [time()]
for datafile, datetime in datafiles_datetimes[1:11]:
    rdd = sc.textFile(path).map(lambda r: [datetime] + r.split(' '))
    df = sql.createDataFrame(rdd, schema=['datetime', 'projectcode', 'page', 'views', 'weight'])
    df.write.format("org.apache.spark.sql.cassandra").\
        options(table="wikipediadata", keyspace="projet").\
        save(mode="append")
    ts.append(time())

In [55]:
with open("log_time", "a") as f:
    for i, t in enumerate(ts[1:]):
        f.write('Chargement du fichier {} : {} secondes.\n'.format(i, t-ts[i]))
    total = ts[-1] - ts[0]
    m = total // 60
    s = total % 60
    f.write('Temps total pour 10 fichiers : {} minutes {} secondes\n'.format(m, s))

On time l'insertion de 3 fichiers via le master, avec concatenation de ces fichiers avant l'écriture dans Cassandra.

On filtre les pages puis on insert dans une df. Et on concatene les df

In [137]:
ts = [time()]
global_df = None
for datafile, datetime in datafiles_datetimes[3:5]:
    path='file:///mnt/wikidata/wikistats/' + datafile 
    rdd = sc.textFile(path).map(lambda r: create_row(r, datetime)).filter(no_points)
    df = sql.createDataFrame(rdd, schema=['datetime', 'projectcode', 'page', 'views', 'weight'])
    if global_df is None:
        global_df = df
    else:
        global_df = global_df.unionAll(df)
    ts.append(time())
global_df.write.format("org.apache.spark.sql.cassandra").\
    options(table="wikidata", keyspace="projet").\
    save(mode="append")
ts.append(time())

In [133]:
with open("log_time", "a") as f:
    for i, t in enumerate(ts[1:]):
        f.write('Chargement du fichier {} : {} secondes.\n'.format(i, t-ts[i]))
    f.write('Ecriture de la dataframe de 10 fichiers {} : {} secondes.'.format(i, ts[-1]-ts[-2]))
    total = ts[-1] - ts[0]
    m = total // 60
    s = total % 60
    f.write('Temps total pour 10 fichiers : {} minutes {} secondes\n'.format(m, s))

In [134]:
global_df.count()

15097106

---

# Request

In [8]:
user = sql.read.format("org.apache.spark.sql.cassandra").\
               load(keyspace="training", table="user")

In [6]:
adults = user.where(user.age > 21)


In [7]:
adults.show()


+-------+---+--------------------+-------+
|user_id|age|      favorite_foods|   name|
+-------+---+--------------------+-------+
|      3|108|ArrayBuffer(Muffi...|Patrick|
|      1| 34|ArrayBuffer(Bacon...|    Jon|
|      2| 22|ArrayBuffer(Kale,...|   Dani|
+-------+---+--------------------+-------+



# Request

In [138]:
wikidata = sql.read.format("org.apache.spark.sql.cassandra").\
               load(keyspace="projet", table="wikidata")

In [63]:
wikidata.printSchema()

root
 |-- datetime: timestamp (nullable = true)
 |-- page: string (nullable = true)
 |-- projectcode: string (nullable = true)
 |-- views: long (nullable = true)



In [139]:
wikidata.groupBy("datetime").count().show()

+--------------------+-------+
|            datetime|  count|
+--------------------+-------+
|2011-01-01 06:00:...|3390237|
|2011-03-13 22:00:...|5706698|
|2011-01-30 07:00:...|4245088|
|2011-01-13 18:00:...|5901278|
|2011-03-05 21:00:...|5816846|
+--------------------+-------+



In [150]:
date = wikidata.filter(wikidata.datetime > '2011-03-05')

In [151]:
date.show()

+--------------------+--------------------+-----------+-----+
|            datetime|                page|projectcode|views|
+--------------------+--------------------+-----------+-----+
|2011-03-05 21:00:...|                   !|         de|    4|
|2011-03-05 21:00:...|                   !|         en|   13|
|2011-03-05 21:00:...|                   !|         fr|    1|
|2011-03-05 21:00:...|                   !|         pl|    1|
|2011-03-05 21:00:...|                   !|         ro|    1|
|2011-03-05 21:00:...|                   !|         ru|    1|
|2011-03-05 21:00:...|                  !!|         de|    3|
|2011-03-05 21:00:...|                  !!|         en|    3|
|2011-03-05 21:00:...|                  !!|         fr|    2|
|2011-03-05 21:00:...|                  !!|         zh|    1|
|2011-03-05 21:00:...|                 !!!|         de|    5|
|2011-03-05 21:00:...|                 !!!|         en|   21|
|2011-03-05 21:00:...|                 !!!|         fr|    2|
|2011-03

In [159]:
dategroup = date.groupBy("datetime", "page")

In [161]:
res = dategroup.agg(F.sum(date.views)).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job 190 cancelled because Stage 250 was cancelled
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1276)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1228)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply$mcVI$sp(DAGScheduler.scala:1216)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1215)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1215)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofInt.foreach(ArrayOps.scala:156)
	at org.apache.spark.scheduler.DAGScheduler.handleStageCancellation(DAGScheduler.scala:

In [103]:
datafiles_datetimes[4:6]

[('pagecounts-20110101-060000.gz', '2011-01-01 06:00'),
 ('pagecounts-20110314-110000.gz', '2011-03-14 11:00')]

In [157]:
# val results = sql.sql("SELECT page FROM wikidata where")

In [154]:
def get_pages_period(date_from, date_to, table):
    result = table.filter((table.datetime > date_from) & (table.datetime < date_to))
    
    return result

In [155]:
test = get_pages_period('2011-03-01', '2011-03-30', wikidata)

In [156]:
test_datetime = test.groupBy("page")

In [142]:
test_datetime.count().show()

+--------+-----+
|datetime|count|
+--------+-----+
+--------+-----+

